# Kerasによるクレーム分類：Pythonディープラーニングライブラリ

このノートブックでは、請求が自動車保険請求である場合は `1` 、住宅保険請求である場合は `0` を予測する請求テキストの分類モデルをトレーニングします。 モデルは、Kerasライブラリを介してTensorFlowを使用し、Long Short-Term Memory（LSTM）リカレントニューラルネットワークと呼ばれるDNNのタイプを使用して構築されます。

このノートブックでは、以下で構成されるテキスト分析プロセスについて説明します。

- GloVe単語埋め込みと単語アナロジーの例
- GloVe単語の埋め込みを使用したトレーニングデータのベクトル化
- LSTMベースの分類モデルの作成とトレーニング
- トレーニング指標をリアルタイムで監視する
- モデルを使用して分類を予測する

## モジュールを準備する

このノートブックは、Kerasライブラリを使用して分類器を作成およびトレーニングします。

In [ ]:
import string
import re
import os
import numpy as np
import pandas as pd
import urllib.request

import tensorflow as tf
import keras
from keras import models, layers, optimizers, regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, LSTM
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

print('Keras version: ', keras.__version__)
print('Tensorflow version: ', tf.__version__)

import azureml.core
from azureml.core import Experiment, Workspace, Run, Datastore
from azureml.core.dataset import Dataset
from azureml.core.compute import ComputeTarget
from azureml.core.model import Model
from azureml.train.dnn import TensorFlow
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails

print("Azure ML SDK version:", azureml.core.VERSION)

import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
%matplotlib inline

**事前トレーニング済みのGloVe単語の埋め込みをダウンロードして、このノートブックにロードしてみましょう。**

これで、サイズが **400,000** ワードの `dictionary`と、辞書内の単語に対応する ` GloVe wordベクトル ` が作成されます。 各単語ベクトルのサイズは50であるため、ここで使用される単語埋め込みの次元は **50** です。

*次のセルの実行には数分かかる場合があります*

In [ ]:
words_list_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
                  'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/glove50d/wordsList.npy')

word_vectors_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
                    'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/glove50d/wordVectors.npy')

word_vectors_dir = './word_vectors'

os.makedirs(word_vectors_dir, exist_ok=True)
urllib.request.urlretrieve(words_list_url, os.path.join(word_vectors_dir, 'wordsList.npy'))
urllib.request.urlretrieve(word_vectors_url, os.path.join(word_vectors_dir, 'wordVectors.npy'))

dictionary = np.load(os.path.join(word_vectors_dir, 'wordsList.npy'))
dictionary = dictionary.tolist()
dictionary = [word.decode('UTF-8') for word in dictionary]
print('Loaded the dictionary! Dictionary size: ', len(dictionary))

word_vectors = np.load(os.path.join(word_vectors_dir, 'wordVectors.npy'))
print ('Loaded the word vectors! Shape of the word vectors: ', word_vectors.shape)

**収縮マップという単語を作成します。 マップはコーパスの縮約を拡張するために使用されます（たとえば、 "can't" は"cannot"になります）。**

In [ ]:
contractions_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
                    'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/glove50d/contractions.xlsx')
contractions_df = pd.read_excel(contractions_url)
contractions = dict(zip(contractions_df.original, contractions_df.expanded))
print('Review first 10 entries from the contractions map')
print(contractions_df.head(10))

## GloVe単語埋め込みを使用したワードアナロジーの例

GloVeは、辞書内の各単語をベクトルとして表します。ワードアナロジーを予測するために単語ベクトルを使用できます。

次の類推を解決する以下の例を参照してください。 **man と woman の関係は、king と"何"の関係に対応するか？**

コサイン類似度は、2つの単語の類似度を評価するために使用される尺度です。 このヘルパー関数は、2つの単語のベクトルを取り、-1〜1の範囲のコサイン類似度を返します。同義語の場合、コサイン類似度は1に近く、反意語の場合、コサイン類似度は-1に近くなります。

In [ ]:
def cosine_similarity(u, v):
    dot = u.dot(v)
    norm_u = np.linalg.norm(u)
    norm_v = np.linalg.norm(v)
    cosine_similarity = dot/norm_u/norm_v
    return cosine_similarity

単語のベクトルを確認してみましょう **man**, **woman**, and **king**

In [ ]:
man = word_vectors[dictionary.index('man')]
woman = word_vectors[dictionary.index('woman')]
king = word_vectors[dictionary.index('king')]
print(man)
print('')
print(woman)
print('')
print(king)

類推を解くには、次の方程式でxを解く必要があります。

**woman – man = x - king**

したがって, **x = woman - man + king**

In [ ]:
x = woman - man + king

**次に、単語ベクトルが上記で計算されたベクトルxに最も近い単語を見つけます。**

計算コストを制限するために、辞書全体を検索するのではなく、可能な回答のリストから最適な単語を特定します。

In [ ]:
answers = ['woman', 'prince', 'princess', 'england', 'goddess', 'diva', 'empress', 
           'female', 'lady', 'monarch', 'title', 'queen', 'sovereign', 'ruler', 
           'male', 'crown', 'majesty', 'royal', 'cleopatra', 'elizabeth', 'victoria', 
           'throne', 'internet', 'sky', 'machine', 'learning', 'fairy']

df = pd.DataFrame(columns = ['word', 'cosine_similarity'])

# Find the similarity of each word in answers with x
for w in answers:
    sim = cosine_similarity(word_vectors[dictionary.index(w)], x)   
    df = df.append({'word': w, 'cosine_similarity': sim}, ignore_index=True)
    
df.sort_values(['cosine_similarity'], ascending=False, inplace=True)

print(df)

**上記の結果から、単語「queen」のベクトルがベクトル「x」に最も類似していることがわかります。**

数値ばかりで分かりにくいでしょうか？

**【補講】**
単語ベクトル表現のさらなる理解のために、ベクトルを可視化してみましょう。
多次元ベクトルを可視化するためには、次元削減という手法を用います。PCA(主成分分析)を使用すると、元のベクトルが持つ情報をなるべく欠損することなく縮約することができます。今回は50次元のデータを2次元に減らします。これを実行すれば、matplotlib散布図を使用して2次元グラフ上にプロットすることができるようになります。

In [ ]:
# ベクトル同士の演算
man = word_vectors[dictionary.index('man')]
woman = word_vectors[dictionary.index('woman')]
king = word_vectors[dictionary.index('king')]
x = woman - man + king

words = ['king', 'man', 'woman', 'queen']
# GloVe 辞書から指定した単語のベクトルを取り出す
vectors = [word_vectors[dictionary.index(word)] for word in words]

# ベクトル演算結果の確認のため、配列の末尾に演算結果xを追加
vectors.append(x)
words.append('x')

# 50次元を2次元に削減
twodim = PCA(n_components=2).fit_transform(vectors)

plt.figure(figsize=(6,6))
# 散布図を作成
plt.scatter(twodim[:, 0], twodim[:, 1], edgecolors='k', c='b')

for label, x, y in zip(words, twodim[:, 0], twodim[:, 1]):
    labeltitle = label + "(" + str(np.round(x, decimals=1)) + "," + str(np.round(y, decimals=1)) + ")"
    plt.annotate(labeltitle, xy=(x, y), xytext=(5, 0), textcoords="offset points")

plt.show()

「王」から「男性」を引いて「女性」を足したとき、それは「女王」となるであろうという類推が、GloVeベクトルの演算で可能になりました。これは何を意味するかというと、単語間の意味的な関係をベクトルで表現することができるということです。この技術の登場によって、単語をニューラルネットワークのインプットにすることができるようになり、機械翻訳や文章の固有表現抽出など幅広い領域に用いられるようになりました。

それではもっと面白い特徴をお見せしましょう。

In [ ]:
# ベクトル同士の演算
man = word_vectors[dictionary.index('man')]
woman = word_vectors[dictionary.index('woman')]
king = word_vectors[dictionary.index('king')]
x = woman - man + king

# GloVe 辞書内に含まれる単語をカテゴリごとに追加（自由に編集してください）
words = ['coffee', 'tea', 'beer', 'wine', 'brandy', 'rum', 'champagne', 
         'spaghetti', 'borscht', 'hamburger', 'pizza', 'falafel', 'sushi', 'meatballs',
         'dog', 'horse', 'cat', 'monkey', 'parrot', 'koala', 'lizard',
         'frog', 'toad', 'monkey', 'ape', 'kangaroo', 'wombat', 'wolf',
         'france', 'germany', 'hungary', 'luxembourg', 'australia', 'japan',
         'homework', 'assignment', 'problem', 'exam', 'test', 'class',
         'school', 'college', 'university', 'institute',
         'king', 'man', 'woman', 'queen']

# GloVe 辞書から指定した単語のベクトルを取り出す
vectors = [word_vectors[dictionary.index(word)] for word in words]

# ベクトル演算結果の確認のため、配列の末尾に演算結果xを追加
vectors.append(x)
words.append('x')

# 50次元を2次元に削減
twodim = PCA(n_components=2).fit_transform(vectors)

plt.figure(figsize=(14,14))
# 散布図を作成
plt.scatter(twodim[:, 0], twodim[:, 1], edgecolors='k', c='b')

for label, x, y in zip(words, twodim[:, 0], twodim[:, 1]):
    labeltitle = label + "(" + str(np.round(x, decimals=1)) + "," + str(np.round(y, decimals=1)) + ")"
    plt.annotate(labeltitle, xy=(x, y), xytext=(5, 0), textcoords="offset points")

plt.show()


似た意味の単語同士がベクトル空間上で近い位置に配置されることがグラフから分かりましたか？これは魔法のようなことです！🧙

## トレーニングデータを準備する

Contoso Ltdは、クレームテキストとして受け取るテキストの例を含む小さなドキュメントを提供しています。 彼らは、これをサンプルクレームごとに1行のテキストファイルで提供しています。

次のセルを実行して、ファイルの内容をダウンロードして確認します。 少し時間をかけてクレームを読んでください（あなたはそれらのいくつかをかなりコミカルに感じるかもしれません！）。

In [ ]:
data_location = './data'
base_data_url = 'https://databricksdemostore.blob.core.windows.net/data/05.03/'
filesToDownload = ['claims_text.txt', 'claims_labels.txt']

os.makedirs(data_location, exist_ok=True)

for file in filesToDownload:
    data_url = os.path.join(base_data_url, file)
    local_file_path = os.path.join(data_location, file)
    urllib.request.urlretrieve(data_url, local_file_path)
    print('Downloaded file: ', file)
    
claims_corpus = [claim for claim in open(os.path.join(data_location, 'claims_text.txt'))]
claims_corpus

クレームサンプルに加えて、Contoso Ltdは、提供されたサンプルクレームのそれぞれに0（「住宅保険クレーム」）または1（「自動車保険クレーム」）のいずれかとしてラベルを付けるドキュメントも提供しています。 これは、サンプルごとに1行のテキストファイルとして提示され、クレームテキストと同じ順序で提示されます。

次のセルを実行して、提供されたClaims_labels.txtファイルの内容を調べます。

In [ ]:
labels = [int(re.sub("\n", "", label)) for label in open(os.path.join(data_location, 'claims_labels.txt'))]
print(len(labels))
print(labels[0:5]) # first 5 labels
print(labels[-5:]) # last 5 labels

上記の出力からわかるように、値は整数0または1です。これらをモデルのトレーニングに使用するラベルとして使用するには、これらの整数値をカテゴリ値に変換する必要があります（他のプログラミングの列挙型のようなものと考えてください） 言語）。

`keras.utils` の to_categorical メソッドを使用して、これらの値をバイナリのカテゴリ値に変換できます。 次のセルを実行します。

In [ ]:
labels = to_categorical(labels, 2)
print(labels.shape)
print()
print(labels[0:2]) # first 2 categorical labels
print()
print(labels[-2:]) # last 2 categorical labels

クレームテキストとラベルが読み込まれたので、テキスト分析プロセスの最初のステップ、つまりテキストを正規化する準備ができました。

### クレームコーパスを処理する

- すべての単語を小文字にします
- 収縮を拡張する（たとえば、 "can't" が "cannot" になる）
- 特殊文字（句読点など）を削除する
- クレームテキスト内の単語のリストを、辞書内のそれらの単語の対応するインデックスのリストに変換します。 書面のクレームに表示される単語の順序は維持されることに注意してください。

次のセルを実行して、クレームコーパスを処理します。

In [ ]:
def remove_special_characters(token):
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
    filtered_token = pattern.sub('', token)
    return filtered_token

def convert_to_indices(corpus, dictionary, c_map, unk_word_index = 399999):
    sequences = []
    for i in range(len(corpus)):
        tokens = corpus[i].split()
        sequence = []
        for word in tokens:
            word = word.lower()
            if word in c_map:
                resolved_words = c_map[word].split()
                for resolved_word in resolved_words:
                    try:
                        word_index = dictionary.index(resolved_word)
                        sequence.append(word_index)
                    except ValueError:
                        sequence.append(unk_word_index) #Vector for unkown words
            else:
                try:
                    clean_word = remove_special_characters(word)
                    if len(clean_word) > 0:
                        word_index = dictionary.index(clean_word)
                        sequence.append(word_index)
                except ValueError:
                    sequence.append(unk_word_index) #Vector for unkown words
        sequences.append(sequence)
    return sequences

claims_corpus_indices = convert_to_indices(claims_corpus, dictionary, contractions)

**1つのサンプルクレームのインデックスを確認する**

In [ ]:
print(remove_special_characters(claims_corpus[5]).split())
print()
print('Ordered list of indices for the above claim')
print(claims_corpus_indices[5])
print('')
print('For example, the index of second word in the claims text \"pedestrian\" is: ', dictionary.index('pedestrian'))

**固定長ベクトルを作成する**

クレームで使用される単語の数は、クレームによって異なります。 固定サイズの入力ベクトルを作成する必要があります。 `keras.preprocessing.sequence` のユーティリティ関数 ` pad_sequences` を使用して、単語インデックスの固定サイズのベクトル（サイズ= 125）を作成します。

In [ ]:
maxSeqLength = 125

X = pad_sequences(claims_corpus_indices, maxlen=maxSeqLength, padding='pre', truncating='post')

print('Review the new fixed size vector for a sample claim')
print(remove_special_characters(claims_corpus[5]).split())
print()
print(X[5])
print('')
print('Lenght of the vector: ', len(X[5]))

### トレーニングデータを blob ストアにアップロードする

**保存された設定ファイルから `Workspace` を作成します。**

In [ ]:
import azureml.core

print(azureml.core.VERSION)

from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print(ws)

**特徴量とラベルをデフォルトのblob ストアにアップロードする**

In [ ]:
input_location = './inputs'
features_file = os.path.join(input_location, 'features.npy')
labels_file = os.path.join(input_location, 'labels.npy')
os.makedirs(input_location, exist_ok=True)
np.save(features_file, X)
np.save(labels_file, labels)

datastore = ws.get_default_datastore()
datastore.upload(input_location, 
                 target_path = 'inputs', 
                 overwrite = True, 
                 show_progress = True)

# Create a file dataset object that represents the features and the labels files
dataset = Dataset.File.from_files((datastore, 'inputs/*.npy'))

## Azure ML Computeを使ってリモートでDNNを訓練する

### AML Compute Clusterの作成

Azure Machine Learning Computeは、機械学習ワークロードを実行するためのAzure仮想マシンのクラスタをプロビジョニングして管理するためのサービスです。現在のワークスペースに新しいAml Computeを作成してみましょう（まだ存在していない場合）。このコンピュートターゲット上でモデルトレーニングジョブを実行します。作成には数分かかります。

In [ ]:
### Create AML CPU based Compute Cluster
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "amlcompute-ad"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC6',
                                                           min_nodes=1, max_nodes=1)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# Use the 'status' property to get a detailed status for the current AmlCompute. 
print(compute_target.status.serialize())

### トレーニングスクリプトの作成

トレーニングスクリプトは、DNNを構築してトレーニングします。ニューラルネットワークの構造を理解するために、以下のコードを確認してください。ご覧のように、このケースではLSTMリカレント・ニューラル・ネットワークを構築します。このネットワークには、単語のインデックスを GloVe 単語ベクトルに変換する単語埋め込み層があります。次に、GloVe単語ベクトルはLSTM層に渡され、続いてバイナリ分類器出力層に渡されます。

次の2つのセルを実行して学習スクリプトを作成し、`scripts`フォルダに保存する。

In [ ]:
script_file_folder = './scripts'
script_file_name = 'train.py'
script_file_full_path = os.path.join(script_file_folder, script_file_name)
os.makedirs(script_file_folder, exist_ok=True)

In [ ]:
%%writefile $script_file_full_path
import os
import argparse
import urllib.request
import math
import timeit
import numpy as np
np.random.seed(437)

print("numpy version: {}".format(np.__version__))

import tensorflow
import keras
from keras.models import Model as KModel #resolve name conflict with Azure Model
from keras import layers, optimizers, regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Embedding
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
from keras.callbacks import Callback

import sklearn
from sklearn.model_selection import train_test_split

from azureml.core import Run
from azureml.core.model import Model

print("keras version: {} tensorflow version: {} sklearn version: {}".format(keras.__version__, 
                                                                            tensorflow.__version__, 
                                                                            sklearn.__version__))

parser = argparse.ArgumentParser("train")
parser.add_argument("--input-folder", type=str, dest='input_folder', help="Reference to inputs")
parser.add_argument('--batch-size', type=int, dest='batch_size', default=16, help='mini batch size for training')
parser.add_argument('--lr', type=float, dest='lr', default=0.0005, help='Learning rate')
parser.add_argument('--epochs', type=int, dest='epochs', default=25, help='Epochs')

args = parser.parse_args()
input_location = args.input_folder
batch_size = args.batch_size
lr = args.lr
epochs = args.epochs

print('Input folder', input_location)
print('Learning rate', lr)
print('Batch size', batch_size)
print('Epochs', epochs)

words_list_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
                  'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/glove50d/wordsList.npy')
word_vectors_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
                    'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/glove50d/wordVectors.npy')
word_vectors_dir = './word_vectors'
os.makedirs(word_vectors_dir, exist_ok=True)
urllib.request.urlretrieve(words_list_url, os.path.join(word_vectors_dir, 'wordsList.npy'))
urllib.request.urlretrieve(word_vectors_url, os.path.join(word_vectors_dir, 'wordVectors.npy'))
dictionary = np.load(os.path.join(word_vectors_dir, 'wordsList.npy'))
dictionary = dictionary.tolist()
dictionary = [word.decode('UTF-8') for word in dictionary]
print('Loaded the dictionary! Dictionary size: ', len(dictionary))
word_vectors = np.load(os.path.join(word_vectors_dir, 'wordVectors.npy'))
print ('Loaded the word vectors! Shape of the word vectors: ', word_vectors.shape)

maxSeqLength = 125
embedding_layer = Embedding(word_vectors.shape[0],
                            word_vectors.shape[1],
                            weights=[word_vectors],
                            input_length=maxSeqLength,
                            trainable=False)

model = Sequential()
model.add(embedding_layer)
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='sigmoid'))
print(model.summary())

print('Reading input files...')
features_file = os.path.join(input_location, 'features.npy')
labels_file = os.path.join(input_location, 'labels.npy')
X = np.load(features_file)
labels = np.load(labels_file)
print('Done reading input files.')

X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=0)

opt = keras.optimizers.Adam(lr=lr)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

run = Run.get_context()
class LogRunMetrics(Callback):
    # callback at the end of every epoch
    def on_epoch_end(self, epoch, log):
        # log a value repeated which creates a list
        run.log('Loss', log['val_loss'])
        run.log('Accuracy', log['val_accuracy'])
_callbacks = [LogRunMetrics()]

print("Model training starting...")
start_time = timeit.default_timer()
history = model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    validation_data=(X_test, y_test),
                    callbacks=_callbacks,
                    verbose=2)
elapsed_time = timeit.default_timer() - start_time
print("Model training completed.")
print('Elapsed time (min): ', round(elapsed_time/60.0,0))

# save the model
os.makedirs('./outputs', exist_ok=True)
model.save(os.path.join('./outputs', 'final_model.hdf5'))
# save training history
with open(os.path.join('./outputs', 'history.txt'), 'w') as f:
    f.write(str(history.history))

print("Model saved in ./outputs folder")
print("Saving model files completed.")

print('Done!')

### Keras estimator を作成します。

引数 **script_params** は、トレーニングスクリプトに渡すコマンドライン引数の辞書であることに注意してください。この場合、以下のパラメータをトレーニングスクリプトに渡します。

- **input-folder**: リモートコンピュートにマウントされたトレーニングデータセットへのパス
- **lr**: オプティマイザの学習率
- **batch-size**: トレーニング用バッチサイズ
- **epochs**: トレーニングエポック数

In [ ]:
script_params = {
    # to mount files referenced by the dataset
    '--input-folder': dataset.as_named_input('inputs').as_mount(),
    '--lr': 0.001,
    '--batch-size': 16,
    '--epochs': 100,
}

keras_est = TensorFlow(source_directory=script_file_folder,
                       compute_target=compute_target,
                       entry_script=script_file_name, 
                       script_params=script_params, 
                       conda_packages=['numpy==1.18.5', 'scikit-learn==0.23.1'], 
                       pip_packages=['keras==2.3.1'], 
                       framework_version='2.0')

### トレーニングランの送信

トレーニング実行をAzureMachineLearningコンピューティングターゲットに送信するためのコードパターンは常に次のとおりです。

- 実行する実験を作成します。
- 実験を送信する。
- 実行が完了するのを待つ。

In [ ]:
experiment_name = 'claims-classification-exp'
experiment = Experiment(ws, experiment_name)
run = experiment.submit(keras_est)

### 実行メトリクスの監視

azureml Jupyter ウィジェットを使用して、トレーニング実行を監視することができます。トレーニングの進行に合わせて、検証精度と検証損失をリアルタイムで監視することができます。

トレーニングは約2-4分で完了します。トレーニングが完了したら、**Download the trained models** セルを実行して、トレーニングされたモデルをローカルにダウンロードすることができます。

In [ ]:
RunDetails(run).show()

Jupyterウィジェットの `Output Logs` ドロップダウンで **azureml-logs/70_driver_log.txt** を選択して、モデルトレーニングの出力を確認します。"val_accuracy" という値の最終出力を見てください。これは検証セットの精度を表しています。ランダムチャンスの精度が50%と考えれば、ランダムよりも良いモデルなのでしょうか？

この時点ではランダムよりも優れていなくても構いません。これはあなたの最初のモデルに過ぎません! 典型的なデータサイエンスのプロセスでは、モデルの精度を向上させるために、以下のようなさまざまなアクションを繰り返していきます。
- 学習のために、より多くのラベル付き文書を取得する
- 過学習を防ぐための正則化
- レイヤー数、レイヤーごとのノード数、学習率などのモデルハイパーパラメータの調整

### 学習したモデルとモデル履歴ファイルをダウンロードする

In [ ]:
local_outputs = './outputs'
os.makedirs(local_outputs, exist_ok=True)

for f in run.get_file_names():
    if f.startswith('outputs'):
        output_file_path = os.path.join(local_outputs, f.split('/')[-1])
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        run.download_file(name=f, output_file_path=output_file_path)

## 分類クレームのテスト

モデルを作成したので、一連のクレームに対して試してみてください。

**訓練されたモデルをロードします**

In [ ]:
from keras.models import load_model
model = load_model(os.path.join(local_outputs, 'final_model.hdf5'))
print('Model loaded!')
print(model.summary())

**以下のセルを実行して、テストデータを準備します。**

In [ ]:
test_claim = ['I crashed my car into a pole.', 
              'The flood ruined my house.', 
              'I lost control of my car and fell in the river.']

test_claim_indices = convert_to_indices(test_claim, dictionary, contractions)
test_data = pad_sequences(test_claim_indices, maxlen=maxSeqLength, padding='pre', truncating='post')

**モデルを使って分類を予測する**

In [ ]:
pred = model.predict(test_data)
pred_label = pred.argmax(axis=1)
pred_df = pd.DataFrame(np.column_stack((pred,pred_label)), columns=['class_0', 'class_1', 'label'])
pred_df.label = pred_df.label.astype(int)
print('Predictions')
pred_df

最後に、もっとディープラーニングについて触りたいという方のために、追加のコンテンツを用意してあります。
このノートブックでは、複数のテキスト分類アルゴリズムを使ってモデルの精度や学習時間、推論速度の比較をすることができます。Kerasで簡単に試すことができるアルゴリズムとして、LSTM のほかに SimpleRNN と GRU があります。

[SimpleRNN/GRU/LSTMの比較](./06%20Compare%20deep%20learning%20algorithms.ipynb)